In [ ]:
# 使用bi-lstm作为基本结构，进行模型预测，采用pytorch实现
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import time 
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SOS_token = 0
EOS_token = 1

MAX_LENGTH = 10

teacher_forcing_ratio = 0.5

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    
import torch.optim as optim

In [90]:
# data preprocess
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {0:'SOS',1:'EOS'}
        self.word2count = {}
        self.n_words = 2 # include SOS and EOS
        
    def addSentence(self, sentence):
        for item in sentence.split(' '):
            self.addWord(item)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
import unicodedata
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


# Lowercase, trim, and remove non-letter characters
import re
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs


# filter pairs to english sentence start with eng_prefixes
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


# prepareData 
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

import random
print(random.choice(pairs))

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    # input tensor shape: [len(pair[0])+1, 1]
    input_tensor = tensorFromSentence(input_lang, pair[0])
    # target tensor shape: [len(pair[0])+1, 1]
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# split train and test dataset
random.shuffle(pairs)
split_ratio = 0.8
split_idx = int(len(pairs) * split_ratio)
train_pairs = [tensorsFromPair(x) for x in pairs[:split_idx]]
test_pairs = [tensorsFromPair(x) for x in pairs[split_idx:]]
print("train_pairs numbers: " + str(len(train_pairs)))
print("test_pairs numbers: " + str(len(test_pairs)))
print(random.choice(train_pairs))


Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['c est une femme jalouse .', 'she s a jealous woman .']
train_pairs numbers: 8479
test_pairs numbers: 2120
(tensor([[ 348],
        [ 349],
        [ 385],
        [ 545],
        [ 270],
        [1237],
        [   5],
        [   1]]), tensor([[221],
        [124],
        [681],
        [ 44],
        [  4],
        [  1]]))


In [20]:
# 这是一个单层单向的encoder GRU单元的实现 
class EncoderGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(input_size = hidden_size, hidden_size = hidden_size)
    
    def forward(self, x, hidden): 
        # x shape: [1] ? 
        # embedded shape: [1,1, hidden_size]
        embedded = self.embedding(x).view(1,1,-1)        # 这里的1是batch_size，1是输入序列的长度，-1是输入序列的维度
        output = embedded
        # hidden shape: [1, 1, hidden_size] for [D∗num_layers, batch_size, hidden_size] 
        # output shape: [1, 1, hidden_size] for [batch_size, sequence_length, hidden_size]
        output, hidden = self.gru(output, hidden)
        return output, hidden 
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
# 这是一个单层单向的decoder GRU单元的实现 
class DecoderGRU(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderGRU,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size) 
        self.gru = nn.GRU(input_size = hidden_size, hidden_size = hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        # input shape: [1]
        # hidden shape: [1, 1, hidden_size] 
        # output shape: [1, 1, hidden_size]        
        output = self.embedding(input).view(1, 1, -1)
        # element-wise relu
        output = F.relu(output)
        # output shape: [1, 1, hidden_size]
        output, hidden = self.gru(output, hidden)
        # output[0] shape: [1, hidden_size]
        # self.out(output[0]) shape: [1, output_size] 
        # self.softmax(self.out(output[0])) shape: [output_size]
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [21]:
# 这是加入了attention机制的一个单层单向的decoder GRU单元的实现 
# Luong Attention 
class AttDecoderGRU(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(AttDecoderGRU,self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size 
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(input_size = hidden_size, hidden_size = hidden_size)
        # used to compute Ht_hat from concated context vector and current hidden state with Wc
        self.linear = nn.Linear(hidden_size*2, hidden_size) 
        # compute output with Ws
        self.out = nn.Linear(hidden_size, output_size) 
        
    def forward(self, input, hidden, encoder_outputs):
        # encoder_outputs shape: [1, max_len, hidden_size] 
        # input shape: [1]
        # embedded shape: [1,1, hidden_size]
        embedded = self.embedding(input).view(1, 1, -1)
        
        # compute gru  
        # output shape: [1, 1, hidden_size] for [batch_size, sequence_length, hidden_size]
        # hidden shape: [1, 1, hidden_size] for [D∗num_layers, batch_size, hidden_size] 
        output, hidden = self.gru(embedded, hidden)
        
        # attention weights = encoder_outputs * output
        # attn_weights shape: [1, max_len, 1]
        attn_weights = F.softmax(torch.bmm(encoder_outputs, output.transpose(1,2)), dim = 1)
        # context vector shape: [1, 1, hidden_size]
        context_vector = torch.bmm(attn_weights.transpose(1,2), encoder_outputs)
        
        # ht_hat shape: [1,1,hidden_size]
        # compute element-wise tanh
        ht_hat = F.tanh(self.linear(torch.cat((context_vector, hidden), dim = 2)))
        
        # output loglikelihood softmax result
        # output shape: [1, 1, output_size]
        # hidden shape: [1, 1, hidden_size]
        output = F.log_softmax(self.out(ht_hat), dim = 2) 
        
        return output, hidden
    
    def initHidden(self): 
        return torch.zeros(1, 1, self.hidden_size, device=device) 

In [102]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, max_length=MAX_LENGTH):
        super(Model, self).__init__()
        self.encoder = EncoderGRU(input_size, hidden_size)
        self.decoder = AttDecoderGRU(hidden_size, output_size)
        self.max_length = max_length
        
    def forward(self, input_tensor):
        # initialize input hidden state, shape:[1,1,hidden_size]
        encoder_hidden = self.encoder.initHidden()
        
        # encoder_outputs shape: [1, max_length, encoder.hidden_size]
        encoder_outputs = torch.zeros(1, self.max_length, self.encoder.hidden_size, device=device)
        
        input_length = input_tensor.size(0)
        # encoder
        for ei in range(input_length):
            # encoder_output shape: [1, 1, encoder.hidden_size]
            encoder_output, encoder_hidden = self.encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[0,ei] = encoder_output[0,0]
            
        # decoder
        decoder_input = torch.tensor([[SOS_token]], device=device)
        # decoder_hidden shape: [1, 1, decoder.hidden_size]
        decoder_hidden = encoder_hidden
        
        decoder_outputs = torch.zeros(1, self.max_length, self.decoder.output_size, device=device) 
        for di in range(self.max_length):
            # decoder_output shape: [1, 1, decoder.output_size]
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            decoder_outputs[0,di] = decoder_output[0,0]
            if decoder_input.item() == EOS_token:
                break
            
        return encoder_outputs, encoder_hidden, decoder_outputs
        
    def train_process(self, n_iters, training_pairs, loss_function, print_every=1000, plot_every=100, learning_rate=0.01):
        # super(Model, self).train(mode=True)
        self.train(mode=True)
        start = time.time()
        train_losses = []
        test_losses = []
        print_loss_total = 0  # Reset every print_every
        plot_loss_total = 0  # Reset every plot_every

        model_optimizer = optim.SGD(self.parameters(), lr=learning_rate) 

        training_pairs = [random.choice(training_pairs) for i in range(n_iters)]
        loss_function = nn.NLLLoss()

        for iter in range(1, n_iters + 1):
            training_pair = training_pairs[iter - 1]
            # input_tensor shape: 
            input_tensor = training_pair[0] 
            # target_tensor shape: 
            target_tensor = training_pair[1]

            # train with input_tensor and target_tensor step 
            # -------
            model_optimizer.zero_grad()
            loss = 0
            target_length = target_tensor.size(0)  
            
            # forward passing 
            # encoder_outputs shape: [1, self.max_length, self.encoder.hidden_size]
            # encoder_hidden shape: [1, 1, self.decoder.hidden_size]
            # decoder_outputs shape:  [1, self.max_length, self.decoder.output_size] 
            encoder_outputs, encoder_hidden, decoder_outputs = self.forward(input_tensor)
            
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            if use_teacher_forcing:
                decoder_hidden = encoder_hidden
                decoder_input = torch.tensor([[SOS_token]], device=device)
                for di in range(target_length):
                    decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
                    loss += loss_function(decoder_output[0,0], target_tensor[di,0])
                    decoder_input = target_tensor[di]  # Teacher forcing
                target_nums = target_length    
            else:
                target_nums = 0
                for di in range(target_length):
                    topv, topi = decoder_outputs[0,di].topk(1)
                    if topi.squeeze().detach().item() == EOS_token:
                        break
                    loss += loss_function(decoder_outputs[0,di], target_tensor[di,0])
                    target_nums +=1
                    

            loss.backward()
            model_optimizer.step()
            
            # ? target_nums应该都是target_length吗
            loss = loss/target_nums

            # -------
            
            print_loss_total += loss
            
#             plot_loss_total += loss
#             if iter % plot_every == 0:
#                 plot_loss_avg = plot_loss_total / plot_every
#                 plot_losses.append(plot_loss_avg)
#                 plot_loss_total = 0

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                train_losses.append(print_loss_avg)
                print_loss_total = 0 
                print("%s, (%d, %d%%), %.4f" % (timeSince(start, iter/n_iters), iter, iter/n_iters*100, print_loss_avg))
                print("is Model in training mode before eval: " + str(self.training))
                test_loss = self.eval(test_pairs, loss_function)
                test_losses.append(test_loss)
                print("is Model in training mode after eval: " + str(self.training))
                # change to train mode mannully
                self.train(mode=True)


        showPlot(train_losses)
        showPlot(test_loss)
    
    # compute test loss over test set
    def eval(self, test_pairs, loss_function, n_iters = len(test_pairs)):
        super(Model, self).eval()
        total_loss = 0 
        test_pairs = [random.choice(test_pairs) for i in range(n_iters)]
        for iter in range(n_iters):
            loss = 0
            input_tensor = test_pairs[iter][0]
            target_tensor = test_pairs[iter][1]
            target_length = target_tensor.size(0)
            encoder_outputs, encoder_hidden, decoder_outputs = self.forward(input_tensor)
            target_nums = 0
            for di in range(target_length):
                topv, topi = decoder_outputs[0,di].topk(1)
                if topi.squeeze().detach().item() == EOS_token:
                    break
                loss += loss_function(decoder_outputs[0,di], target_tensor[di,0])
                target_nums += 1
                
                
            loss /= target_nums
            total_loss += loss
            
        total_loss /= n_iters
        print("test set loss: %.4f" % total_loss)
        return total_loss 
        

In [ ]:
# start training using model
hidden_size = 256
seq2seq_model = Model(input_lang.n_words, hidden_size, output_lang.n_words, 10).to(device)
loss_function = nn.NLLLoss()
seq2seq_model.train_process(75000, train_pairs, loss_function, print_every=1000)

0m 22s (- 27m 37s), (1000, 1%), 3.9143
is Model in training mode before eval: True
test set loss: 3.8179
is Model in training mode after eval: False
1m 38s (- 60m 12s), (2000, 2%), 3.3631
is Model in training mode before eval: True
test set loss: 3.3109
is Model in training mode after eval: False
2m 18s (- 55m 12s), (3000, 4%), 2.9126
is Model in training mode before eval: True
test set loss: 3.1306
is Model in training mode after eval: False
3m 2s (- 53m 57s), (4000, 5%), 2.8081
is Model in training mode before eval: True
test set loss: 3.1055
is Model in training mode after eval: False
3m 53s (- 54m 31s), (5000, 6%), 2.6547
is Model in training mode before eval: True
test set loss: 2.9432
is Model in training mode after eval: False
4m 35s (- 52m 51s), (6000, 8%), 2.5391
is Model in training mode before eval: True
test set loss: 2.9591
is Model in training mode after eval: False
5m 20s (- 51m 56s), (7000, 9%), 2.4926
is Model in training mode before eval: True
test set loss: 2.8207
is

In [44]:
# Print model's state_dict
print("Model's state_dict:")
parameters_nums = 0
for param_tensor in seq2seq_model.state_dict():
    print(param_tensor, "\t", seq2seq_model.state_dict()[param_tensor].size())
    parameters_nums += seq2seq_model.state_dict()[param_tensor].numel()
    
print("parameter number is: " + str(parameters_nums))
    
# # Print optimizer's state_dict
# print("Optimizer's state_dict:")
# for var_name in optimizer.state_dict():
#     print(var_name, "\t", optimizer.state_dict()[var_name])

# save model
torch.save(seq2seq_model.state_dict(), 'models/seq2seq_model.pth')

Model's state_dict:
encoder.embedding.weight 	 torch.Size([4345, 256])
encoder.gru.weight_ih_l0 	 torch.Size([768, 256])
encoder.gru.weight_hh_l0 	 torch.Size([768, 256])
encoder.gru.bias_ih_l0 	 torch.Size([768])
encoder.gru.bias_hh_l0 	 torch.Size([768])
decoder.embedding.weight 	 torch.Size([2803, 256])
decoder.gru.weight_ih_l0 	 torch.Size([768, 256])
decoder.gru.weight_hh_l0 	 torch.Size([768, 256])
decoder.gru.bias_ih_l0 	 torch.Size([768])
decoder.gru.bias_hh_l0 	 torch.Size([768])
decoder.linear.weight 	 torch.Size([256, 512])
decoder.linear.bias 	 torch.Size([256])
decoder.out.weight 	 torch.Size([2803, 256])
decoder.out.bias 	 torch.Size([2803])
parameter number is: 3471091


In [49]:
# load model
loaded_model = Model(input_lang.n_words, hidden_size, output_lang.n_words, 10)
loaded_model.load_state_dict(torch.load('models/seq2seq_model.pth'))

loss_function = nn.NLLLoss()
loaded_model.train(75000, loss_function, print_every=1000)

1m 3s (- 77m 58s), (1000, 1%), 0.6060
1m 34s (- 57m 42s), (2000, 2%), 0.6004


KeyboardInterrupt: 

In [7]:
evaluateRandomly(encoder1, attn_decoder1)

NameError: name 'evaluateRandomly' is not defined